<a href="https://colab.research.google.com/github/1xuan-wu/1141-Programming-Language-/blob/main/41171220H_HW1_%E6%97%A5%E5%B8%B8%E6%94%AF%E5%87%BA%E9%80%9F%E7%AE%97%E8%88%87%E5%88%86%E6%94%A4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#日常支出速算與分攤（作業一）
- 目標：從 Sheet 讀「消費紀錄」→ 計總額/分類小計/AA 分攤 → 寫回 Sheet Summary 分頁。
- AI 點子（可選）：請模型總結本週花錢習慣與建議（例如「外食過多」）。
- Sheet 欄位：date, category, item, amount, payer

GoogleSheet: https://docs.google.com/spreadsheets/d/1uRehhGB9piBOMHsZS0Pi6PSkN0NB2GGKaMs2J92ZTK0/edit?gid=0#gid=0

In [123]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [124]:
import pandas as pd
# read data and put it in a dataframe
# 在 google 工作表載入 gsheets
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1uRehhGB9piBOMHsZS0Pi6PSkN0NB2GGKaMs2J92ZTK0/edit?gid=0#gid=0')

In [125]:
# 從 gsheets 的 消費紀錄 載入 sheets
sheets = gsheets.worksheet('消費紀錄').get_all_values()

# 將 sheets 資料載入 pd 的 DataFrame 進行分析
# Assuming the first row is headers, read data from the second row onwards
# And manually set column names to match the expected headers.
# Based on the sheet structure, the columns should be '日期', '時間', '品項', '金額', 'Category', 'Payer'.
if sheets:
    # Use the first row as headers
    headers = sheets[0]
    df = pd.DataFrame(sheets[1:], columns=headers)

    # Display the first few rows and columns to understand the data structure
    display(df.head())
    display(df.columns)
else:
    print("Google Sheet is empty.")

,日期,時間,品項,金額,,總支出
0,2025-09-18,12:04,早餐,100,,500
1,2025-09-08,12:08,午餐,200,,
2,2025-09-18,12:13,下午茶,100,,
3,2025-09-18,12:20,水,20,,
4,2025-02-18,12:25,水,20,,


Index(['日期', '時間', '品項', '金額', '', '總支出'], dtype='object')

In [126]:
import datetime

In [127]:
# 讓使用者輸入資料
date_str = input("請輸入日期 (格式：YYYY-MM-DD): ")
# 檢查日期格式是否正確
datetime.datetime.strptime(date_str, '%Y-%m-%d')

time_str = input("請輸入時間 (格式：HH:MM): ")
# 檢查時間格式是否正確
datetime.datetime.strptime(time_str, '%H:%M')

item = input("請輸入品項: ")
amount = float(input("請輸入金額: "))

# 創建一個新的 DataFrame 來代表你要新增的資料
new_data = pd.DataFrame([{
    '日期': date_str,
    '時間': time_str,
    '品項': item,
    '金額': amount
}])

# 使用 concat() 將新資料合併到舊的 df 中
# Assuming df already contains data from the sheet with correct columns
# Let's first ensure df has the correct structure before concatenating
# This part needs to be done after reading the data and cleaning column names
# For now, let's just keep the new_data creation here. The concatenation
# will be moved to after the data is read and processed from the sheet.
# df = pd.concat([df, new_data], ignore_index=True)

# The new_data will be appended to the sheet first, then the df will be reread
# and the total calculated. This is a safer approach than modifying df in place
# and then trying to sync it back, especially with potential header issues.

請輸入日期 (格式：YYYY-MM-DD): 2025-12-12
請輸入時間 (格式：HH:MM): 12:12
請輸入品項: 稅
請輸入金額: 500


In [128]:
# 取得「消費紀錄」工作表物件
worksheet = gsheets.worksheet('消費紀錄')

# 將總支出寫入 F2 儲存格
# update 方法需要一個 list of lists 作為值
# Convert total_expenditure to float before writing to sheet
worksheet.update('F2', [[float(total_expenditure)]])

print(f"總支出 {total_expenditure} 已寫入 Google Sheet 的 F2 儲存格。")

總支出 500 已寫入 Google Sheet 的 F2 儲存格。


/tmp/ipython-input-564569316.py:7: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('F2', [[float(total_expenditure)]])


In [129]:
date_str

'2025-12-12'

In [130]:
time_str

'12:12'

In [131]:
item

'稅'

In [132]:
amount

500.0

In [133]:
# This cell was previously used for concatenating new_data to df.
# Since the process will now be: read sheet -> add new row to sheet -> reread sheet -> calculate total,
# this step of concatenating new_data to df here is no longer needed immediately after input.
# The new_data DataFrame is still needed for writing to the sheet.
# Keep the new_data DataFrame definition in the input cell (MN3G7hp7ZWvz)
# and use it in the sheet writing cell (pBw3mmWzcJyA).
# This cell can be removed or its content moved if necessary.
# The new_data DataFrame is already created in cell MN3G7hp7ZWvz and used in cell pBw3mmWzcJyA.
# This cell is no longer necessary and can be removed.

In [134]:
# 將 '金額' 欄位轉換為數值型別，errors='coerce' 會將無法轉換的值設為 NaN
df['金額'] = pd.to_numeric(df['金額'], errors='coerce')

# 計算 '金額' 欄位的總和，並排除 NaN 值
total_expenditure = df['金額'].sum()

# 顯示總支出
print(f"總支出：{total_expenditure}")

總支出：500


In [135]:
df

,日期,時間,品項,金額,,總支出
0,2025-09-18,12:04,早餐,100,,500
1,2025-09-08,12:08,午餐,200,,
2,2025-09-18,12:13,下午茶,100,,
3,2025-09-18,12:20,水,20,,
4,2025-02-18,12:25,水,20,,
5,2025-09-18,12:20,水,20,,
6,2025-09-18,12:20,水,20,,
7,2025-12-12,12:12,水,20,,


In [136]:
# 將 DataFrame 轉換成列表的列表 (list of lists)，這是 gspread 支援的資料格式
# We only need to write the new data to the sheet, not the whole dataframe
data_to_write = new_data.values.tolist()

# 第一步：取得工作表物件
worksheet = gsheets.worksheet('消費紀錄')

# 第二步：將資料寫入工作表
# Find the first empty row to append the new data
# This is a more robust way to append data than relying on finding the last row manually.
worksheet.append_rows(values=data_to_write, value_input_option='USER_ENTERED')
print("資料已成功寫入 Google 消費紀錄！")

# After writing the new data, reread the entire sheet to get the updated data
sheets = gsheets.worksheet('消費紀錄').get_all_values()

if sheets:
    # Use the first row as headers
    headers = sheets[0]
    df = pd.DataFrame(sheets[1:], columns=headers)

    # Convert '金額' to numeric
    df['金額'] = pd.to_numeric(df['金額'], errors='coerce')

    # Now calculate the total expenditure from the updated df
    total_expenditure = df['金額'].sum()

    # 将总支出写回 Google Sheet, assuming F2 is the correct cell for total expenditure
    # The update method in gspread requires a list of lists
    worksheet.update('F2', [[float(total_expenditure)]])

    print(f"總支出 {total_expenditure} 已寫入 Google Sheet 的 F2 儲存格。")
else:
    print("Google Sheet is empty after appending, unable to calculate total expenditure.")

資料已成功寫入 Google 消費紀錄！
總支出 1000 已寫入 Google Sheet 的 F2 儲存格。


/tmp/ipython-input-1331664120.py:30: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('F2', [[float(total_expenditure)]])
